In [1]:
import requests
from ipywidgets import interact_manual
from IPython.display import display, HTML, Image
import folium
import itertools

# throughout this entire project I used the help of a resource called json viewer (http://jsonviewer.stack.hu/)
# this helped me massively in taking my raw json text and seeing it in a slightly more digestable way
# I also used https://htmlcolorcodes.com/color-picker/ for my html displays to get colors
# I also used https://ipywidgets.readthedocs.io/en/latest/examples/Using%20Interact.html throughout to understand how the interact works a little better
# Additionally I used the IPython display documentation for my headers and html formatting (https://ipython.readthedocs.io/en/stable/api/generated/IPython.display.html)
# most of my work was done in files within the project folder called 'main', 'spoonacular api', and 'reverse geociding testing'

# get recipe data from list of ingredients
# used https://spoonacular.com/food-api/docs to understand and implement the spoonacular api
# specifically refrenced the find by ingredients documentation (https://spoonacular.com/food-api/docs#Search-Recipes-by-Ingredients)
def get_recipes_data(ingredients, number=1):
    url = "https://api.spoonacular.com/recipes/findByIngredients?apiKey=????????????????????"
    querystring = {'ingredients': ingredients,'number': number,'format': 'json', 'ranking': '2'}
    response = requests.get(url, params=querystring)
    response.raise_for_status
    data = response.json()
    return data

# extract every recipe title from the recipes json
def get_recipe_titles(raw_json):
    titles = []
    for recipe in raw_json:
        title = recipe['title']
        titles.append(title)
    return titles

# retreive lat and lon from inputted location
# reviewed https://www.geeksforgeeks.org/python-try-except/ to refresh myself on try and except and how to use it
# used the nominatim documentation to understand the parameters more in depth (https://nominatim.org/release-docs/latest/api/Search/)
# note: I also consulted the visualization HW to see a good example of this api call (for the ufo geocoding)
def geocode(location):
    url = 'https://nominatim.openstreetmap.org/search'  # base URL without paramters after the "?"
    options = { 'q' : location, 'format' : 'json'}
    response = requests.get(url, params = options)
    response.raise_for_status
    geodata = response.json()

    try:
        lat = geodata[0]['lat']
        lon = geodata[0]['lon']
    except IndexError:
        print('Please enter a valid location')
        print('Your location has been set to the default location of Washinton DC')
        lat = '38.8950368'
        lon = '-77.0365427'

    return lat, lon


# uses a reverse geocode from lat and lon and returns a postal code from the reverse geocode data
# this postalcode will be used as an input in our target store locator
# I used the nominatim reverse documentation here to understand the parameters and what is returned (https://nominatim.org/release-docs/latest/api/Reverse/)
def geo_to_postcode(lat, lon):
    url = 'https://nominatim.openstreetmap.org/reverse'
    options = {'format': 'json', 'lat': lat, 'lon': lon}
    response = requests.get(url, params = options)
    response.raise_for_status
    geodata = response.json()
    return geodata['address']['postcode']

# checks each recipe in our json to see if it matches our title from the dropdown
# returns the recipe id of the recipe selected in the dropdown
# I used this to retreive one specific recipe id, versus taking all of the recipe ids from the raw recipes json
# this is so we can work with just one specific recipe (the one we selected in the dropdown)
def get_selected_recipe_id(raw_json, title):
    for recipe in raw_json:
        if title == recipe['title']:
            return recipe['id']

# similar to the previous function, this retreives all of the json data on the ONE singular recipe that we selected in our dropdown
def get_selected_recipe_data(raw_json, title):
    for recipe in raw_json:
        if title == recipe['title']:
            return recipe


# gather recipe steps from a recipe id
# this uses a seperate api call to get step by step instructions given a recipe id
# I consulted spoonaculars documentation to find and implement this api call (https://spoonacular.com/food-api/docs#Get-Analyzed-Recipe-Instructions)
def get_recipe_steps(id_number):
    url = f"https://api.spoonacular.com/recipes/{id_number}/analyzedInstructions?apiKey=?????????????????"
    querystring = {'format': 'json'}
    response = requests.get(url, params = querystring)
    response.raise_for_status
    recipeSteps = response.json()
    return recipeSteps

# take the steps json and retrieve each step and its respective step number
# loops over each step and the details of each step to nicely print out instructions
def display_steps(data):
    step_num_list = []
    step_desc_list = []
    for step in data[0]['steps']:

        step_numbers = step['number']
        print(step_numbers)

        step_desc = step['step']
        print(step_desc)

        step_num_list.append(step_numbers)
        step_desc_list.append(step_desc)

    # stores in two lists if needed (my program has not used these)
    return step_num_list, step_desc_list

# retreives each used ingredient and prints it
# I did a simple print with no return as we wont need the actual list other than when displaying it
def show_used_ingredients(our_data):
    for ingredient in our_data['usedIngredients']:
        print( ingredient['name'] )


# retrives a list of missing ingredients for the recipe from the user selected recipe
# simply iterates over each missing thing and adds it to a list until there are no more items to loop over
def get_missing_ingredients(our_data):
    missing_list = []
    for ingredient in our_data['missedIngredients']:
        missing_list.append(ingredient['name'])
    return missing_list

# uses an api post to send a payload of our ingredients and return a json with each upc code(barcode) for each ingredient
# I consulted the spoonacular documentation for this as I was not super expereinced with api posts https://spoonacular.com/food-api/docs#Map-Ingredients-to-Grocery-Products
# I initially was going to use this barcode data to find stores with the EXACT item, however the UPC apis were all super expensive so I scrapped this in my main program
def get_barcode(ingredient_list):
    url = "https://api.spoonacular.com/food/ingredients/map?apiKey=?????????????????"
    string_list = str(ingredient_list)
    payload = ' {"ingredients":' + string_list + '}'

    response = requests.post(url, data = payload)
    response.raise_for_status
    barcodeData = response.json()
    return barcodeData

# retreives target locations data in a radius around our entered zip code
# uses the header get request format that the site rapidapi requires (rapidapi.com); which is where I found this api
# the documentation I consulted for this api is at https://rapidapi.com/logicbuilder/api/target-com-store-product-reviews-locations-data/
# this is the only time I needed this header format as I used the parent site for all the other apis rather than rapidapi
def get_target_locations(postalcode, radius="5"):
    url = "https://target-com-store-product-reviews-locations-data.p.rapidapi.com/location/search"
    querystring = {"zip":postalcode,"radius":radius, 'format': 'json'}

    headers = {
    'x-rapidapi-host': "target-com-store-product-reviews-locations-data.p.rapidapi.com",
    'x-rapidapi-key': "???????????????????"
    }

    response = requests.get(url, headers=headers, params=querystring)
    response.raise_for_status
    data = response.json()
    return data

# uses our target store location data and returns a list of lists with each set of coords for each store
# specifically, we make a list containing each lat,lon pair, and add it to our parent list and return that
def get_target_coordinates(targetdata):
    complete_coords_list = []
    for location in targetdata['locations']:
        coords = []
        latitude = location['geographic_specifications']['latitude']
        longitude = location['geographic_specifications']['longitude']
        coords.append(latitude)
        coords.append(longitude)
        complete_coords_list.append(coords)
    return(complete_coords_list)

# get a list containing strings of each full address for every target in our target data
# I had to sift through the json data from the target api where I realized that there is no singular address string
# I had to create my own string using the individual address elements in the json data
def get_address_list(targetdata):
    addresses = []
    for target in targetdata['locations']:
        address_line1 = target['address']['address_line1']
        city = target['address']['city']
        state = target['address']['state']
        post_code = target['address']['postal_code']
        address_string = f"{address_line1}, {city}, {state} {post_code}"
        addresses.append(address_string)
    return addresses

# creates a folium map given coordinates and the optional parameter of zoom level
# I consulted the visualization homework on ufos to better understand folium implementation
# I also used the folium documentation to add to my understanding http://python-visualization.github.io/folium/quickstart.html
def create_map(lat, lon, zoom = 10):
    myMap = folium.Map(location = [lat,lon], zoom_start = zoom)
    return myMap

# this function adds markers to our folium map and returns the marked map (http://python-visualization.github.io/folium/quickstart.html)
# this is one of the more complex functions in my program; I needed to loop over my coordinates for the location parameter
# but also loop over the address list for the popups parameter and this had to be done in a parallel structure so I needed an extra resource
# I consulted the internet (https://www.geeksforgeeks.org/python-iterate-multiple-lists-simultaneously/) and was able to find out how to use the
# zip  method from the itertools module which allows for concurrent looping over two lists
# I could now create a marker and apply both my parameters at once because both lists are always going to be the same length(the coords list and the address list both loop over the same thing in our json)
def add_markers(folium_map, coordlist, address_list):
    tooltip = "Click here to view this Target location's address"
    for (coordinates, address) in zip(coordlist, address_list):
        folium.Marker(coordinates, popup = address, icon = folium.Icon(color="red"), tooltip = tooltip).add_to(folium_map)
    return folium_map

# main program
display(HTML("<h1 style=color:#FF5733;>My Local Kitchen</h1>"))

# gather inputs using interact
@interact_manual(Location='enter a location', Ingredients='flour, butter, sugar')
def main(Location, Ingredients):

    # gets our location as a list to print in our program
    location = geocode(Location)

    # creates individual variables for lat and lon to be used in our reverse geocode
    lat, lon = geocode(Location)

    # uses our lat and lon variables to reverse geocode and get the postal code
    zipcode = geo_to_postcode(lat, lon)

    # prints our location coordinates and our zipcode
    print(f"You are located at {location}, and your postal code is {zipcode}.")


    # gather recipe json file with spoonacular api
    recipe_data = get_recipes_data(Ingredients, 2)

    # gets a list of titles from our recipes
    recipeTitles = get_recipe_titles(recipe_data)

    # creates an interact dropdown to select which recipe to use
    @interact_manual(Recipe=recipeTitles)
    def main2(Recipe):
        # html header
        display(HTML(f"<p1>You have selected: {Recipe}</p1>"))

        # finds our selcted recipe id given what we selected in the dropdown
        recipe_id = get_selected_recipe_id(recipe_data, Recipe)

        # sets up our recipe steps data from our recipe id
        steps_json = get_recipe_steps(recipe_id)

        # show our header
        # use our show steps function to display our steps from our steps json file
        display(HTML("<h2 style=color:#C70039;font-size:90%;>Instructions:</h2>"))
        display_steps(steps_json)

        # gather the json data for the recipe we selected specifically
        # show the ingredients the user has
        our_data = get_selected_recipe_data(recipe_data, Recipe)
        display(HTML("<h3 style=color:#C70039;font-size:90%;>Ingredients you have:</h3>"))
        show_used_ingredients(our_data)

        # uses the json data for our individual recipe to build a list of missing items
        # defines the list in our main program and displays each item
        display(HTML("<h4 style=color:#C70039;font-size:90%;>Ingredients you must buy:</h4>"))
        missingList = get_missing_ingredients(our_data)
        for item in missingList:
            print(item)

        # gets our target location json data from our the postal code that we reverse geocoded to retreive
        targetData = get_target_locations(zipcode)

        # retreives our list of full addresses from the raw target json data
        address_list = get_address_list(targetData)

        # defines a list of coordinates of store locations to be used on our map for markers
        coordList = get_target_coordinates(targetData)

        display(HTML("<h5 style=color:#FB1010;font-size:140%;>Target store locations that may have your missing item:</h5>"))

        # creates a folium map centered at our geocoded location
        myMap = create_map(lat,lon)

        # places the markers down on our folium map at each coordinate set, and adds a popup containing the full address
        marked_map = add_markers(myMap, coordList, address_list)

        display(marked_map)






ModuleNotFoundError: No module named 'requests'